In [1]:
import numpy as np
import sympy as sp
from typing import List, Tuple, Dict, Any, Set
from scipy.stats import levy
import matplotlib.pyplot as plt

# OG:

In [ ]:
import sympy as sp
def padded_binary(i: int, n: int) -> str:
    return bin(i)[2:].zfill(n)

# Replace this method in your class
@staticmethod
def binary_string_to_array(string: str) -> np.ndarray:
    # use a comprehension so we don't rely on the builtin `list` name being callable
    return np.array([int(ch) for ch in string], dtype=int)

def calculate_valid_transitions(n: int, sequential: bool):


    if sequential == True:

        num_states = n + 1
        all_states = [i for i in range(num_states)]        
        # print(f"Total number of states: {num_states}")
        # print("Valid single-step transitions:")

        valid_difference_vectors = set()
        
        valid_X_reactions = [] # distributively
        valid_Y_reactions = [] # distributively


        for i in all_states:
            for j in all_states:
                        # Do not consider transitions from a state to itself
                # if i == j:
                #     continue   
                if j - i == 1:
                    print(f'X reaction from {all_states[i]} -> {all_states[j]}')
                    element = 'X'
                    valid_X_reactions.append([str(i), str(j), i, j, element])

                if j - i == -1:
                    print(f'Y reaction from {all_states[i]} -> {all_states[j]}')
                    element = 'Y'
                    valid_Y_reactions.append([str(i), str(j), i, j, element])



                    # if element == "X":
                    #     print(f"{all_states[i]} --> {all_states[j]} ({element}), {i} -> {j}")
                    # if element == "Y":
                    #     print(f"{all_states[i]} --> {all_states[j]} ({element}), {i} -> {j}")

                    # valid_difference_vectors.add(tuple(binary_string_to_array(all_states[j]) - binary_string_to_array(all_states[i])))

    if sequential == False:

        num_states = 2**n
        all_states = [padded_binary(i, n) for i in range(num_states)]
        
        # print(f"Total number of states: {num_states}")
        # print("Valid single-step transitions:")

        valid_difference_vectors = set()
        
        valid_X_reactions = [] # distributively
        valid_Y_reactions = [] # distributively

        for i in range(num_states):
            for j in range(num_states):
                # Do not consider transitions from a state to itself
                if i == j:
                    continue

                if np.sum(np.abs(binary_string_to_array(all_states[j]) - binary_string_to_array(all_states[i]))) == 1:
                    # Determine if it's a phosphorylation or dephosphorylation event
                    # A +1 indicates phosphorylation, a -1 indicates dephosphorylation
                    element = "X" if np.any(binary_string_to_array(all_states[j]) - binary_string_to_array(all_states[i]) == 1) else "Y"
                    
                    if element == "X":
                        print(f"{all_states[i]} --> {all_states[j]} ({element}), {i} -> {j}")
                        valid_X_reactions.append([str(i), all_states[j], i, j, element])
                    if element == "Y":
                        print(f"{all_states[i]} --> {all_states[j]} ({element}), {i} -> {j}")
                        valid_Y_reactions.append([all_states[i], all_states[j], i, j, element])

                    valid_difference_vectors.add(tuple(binary_string_to_array(all_states[j]) - binary_string_to_array(all_states[i])))

    return valid_X_reactions, valid_Y_reactions

def build_phos_odes(n, prefix=""):
    """
    Build symbolic ODEs for the phosphorylation system for given n (N = 2**n).
    Returns a dict with keys:
      adot, bdot, cdot, xdot, ydot,
    """
    N = 2**n
    a_syms = sp.symbols([f"{prefix}a_{i}" for i in range(N)])
    b_syms = sp.symbols([f"{prefix}b_{i}" for i in range(N - 1)])
    c_syms = sp.symbols([f"{prefix}c_{i}" for i in range(N - 1)])
    x_sym, y_sym = sp.symbols(f"{prefix}x {prefix}y")

    a = sp.Matrix(N, 1, lambda i,j: a_syms[i])
    b = sp.Matrix(N, 1, lambda i,j: b_syms[i])
    c = sp.Matrix(N, 1, lambda i,j: c_syms[i])
    
    x = x_sym
    y = y_sym

    kplus = sp.symbols([f"{prefix}k^+_{i}" for i in range(N)]); kplus[-1] = sp.Integer(0)
    kminus = sp.symbols([f"{prefix}k^-_{i}" for i in range(N)]); kminus[-1] = sp.Integer(0)
    pplus = sp.symbols([f"{prefix}p^+_{i}" for i in range(N)]); pplus[0] = sp.Integer(0)
    pminus = sp.symbols([f"{prefix}p^-_{i}" for i in range(N)]); pminus[0] = sp.Integer(0)

    Kp = sp.diag(*kplus)
    Km = sp.diag(*kminus)
    Pp = sp.diag(*pplus)
    Pm = sp.diag(*pminus)

    # compute allowed transitions
    valid_X_reactions, valid_Y_reactions = calculate_valid_transitions(n, False)
    
    # Build Alpha and Beta with symbols only at allowed (j,k)
    Alpha = sp.zeros(N, N)
    Beta  = sp.zeros(N, N)
    # create symbols for allowed entries
    for (_, _, j,k, _) in valid_X_reactions:
        Alpha[j,k] = sp.symbols(f"{prefix}alpha_{j}_{k}")
    for (_, _, j,k, _) in valid_Y_reactions:
        Beta[j,k] = sp.symbols(f"{prefix}beta_{j}_{k}")
    print(Alpha)
    print(Beta)
    ones = sp.Matrix([1]*N)
    Alpha_row_sums = Alpha * ones
    Beta_row_sums = Beta * ones
    DAlpha = sp.diag(*[Alpha_row_sums[i, 0] for i in range(N)])
    DBeta  = sp.diag(*[Beta_row_sums[i, 0] for i in range(N)])
    print(DAlpha)
    print(DBeta)
    adot = Km * b + Pm * c + Alpha.T * b + Beta.T * c - x * (Kp * a) - y * (Pp * a)
    bdot = x * (Kp * a) - Km * b - DAlpha * b
    cdot = y * (Pp * a) - Pm * c - DBeta * c
    bdot[-1] = sp.Integer(0)
    cdot[0] = sp.Integer(0)
    kplus_vec = sp.Matrix(kplus)
    pplus_vec = sp.Matrix(pplus)

    xdot = - x * (kplus_vec.T * a)[0] + (ones.T * (Km + DAlpha) * b)[0]
    ydot = - y * (pplus_vec.T * a)[0] + (ones.T * (Pm + DBeta) * c)[0]
    # print(type(adot[0]))

    return {
        "adot": sp.expand(adot), "bdot": sp.expand(bdot), "cdot": sp.expand(cdot),
        "xdot": sp.expand(xdot), "ydot": sp.expand(ydot),
    }

# Example: build symbolic ODEs for n=2 (N=4)
n = 2
odes = build_phos_odes(n)
odes["adot"][0]
# sp.pprint(odes["adot"])
# sp.pprint(odes["bdot"])
# sp.pprint(odes["cdot"])
# sp.pprint(odes["xdot"])
# sp.pprint(odes["ydot"])

# ADJUSTED:

In [ ]:
import sympy as sp
import numpy as np
from sympy import Sum

def padded_binary(i: int, n: int) -> str:
    return bin(i)[2:].zfill(n)

@staticmethod
def binary_string_to_array(string: str) -> np.ndarray:
    return np.array([int(ch) for ch in string], dtype=int)

def calculate_valid_transitions(n: int, sequential: bool):
    if sequential == True:
        num_states = n + 1
        all_states = [i for i in range(num_states)]        
        
        valid_difference_vectors = set()
        valid_X_reactions = []
        valid_Y_reactions = []

        for i in all_states:
            for j in all_states:
                if j - i == 1:
                    print(f'X reaction from {all_states[i]} -> {all_states[j]}')
                    element = 'X'
                    valid_X_reactions.append([str(i), str(j), i, j, element])

                if j - i == -1:
                    print(f'Y reaction from {all_states[i]} -> {all_states[j]}')
                    element = 'Y'
                    valid_Y_reactions.append([str(i), str(j), i, j, element])

    if sequential == False:
        num_states = 2**n
        all_states = [padded_binary(i, n) for i in range(num_states)]
        
        valid_difference_vectors = set()
        valid_X_reactions = []
        valid_Y_reactions = []

        for i in range(num_states):
            for j in range(num_states):
                if i == j:
                    continue

                if np.sum(np.abs(binary_string_to_array(all_states[j]) - binary_string_to_array(all_states[i]))) == 1:
                    element = "X" if np.any(binary_string_to_array(all_states[j]) - binary_string_to_array(all_states[i]) == 1) else "Y"
                    
                    if element == "X":
                        print(f"{all_states[i]} --> {all_states[j]} ({element}), {i} -> {j}")
                        valid_X_reactions.append([str(i), all_states[j], i, j, element])
                    if element == "Y":
                        print(f"{all_states[i]} --> {all_states[j]} ({element}), {i} -> {j}")
                        valid_Y_reactions.append([all_states[i], all_states[j], i, j, element])

                    valid_difference_vectors.add(tuple(binary_string_to_array(all_states[j]) - binary_string_to_array(all_states[i])))

    return valid_X_reactions, valid_Y_reactions

def build_phos_odes(n, prefix=""):
    """
    Build symbolic ODEs for the phosphorylation system for given n (N = 2**n).
    Now constructs matrices in a manner more similar to the numpy version.
    """
    N = 2**n
    
    # Define symbols
    a_syms = sp.symbols([f"{prefix}a_{i}" for i in range(0, N)])
    b_syms = sp.symbols([f"{prefix}b_{i}" for i in range(0, N - 1)])
    c_syms = sp.symbols([f"{prefix}c_{i}" for i in range(1, N)])
    x_sym, y_sym = sp.symbols(f"{prefix}x {prefix}y")


    a = sp.Matrix(N, 1, lambda i,j: a_syms[i])
    b = sp.Matrix(N-1, 1, lambda i,j: b_syms[i])
    c = sp.Matrix(N-1, 1, lambda i,j: c_syms[i])
    print(a)
    print(b)
    print(c)
    x = x_sym
    y = y_sym

    # Rate constants
    kplus = sp.symbols([f"{prefix}k^+_{i}" for i in range(0, N-1)])
    kminus = sp.symbols([f"{prefix}k^-_{i}" for i in range(0, N-1)])
    pplus = sp.symbols([f"{prefix}p^+_{i}" for i in range(1, N)])
    pminus = sp.symbols([f"{prefix}p^-_{i}" for i in range(1, N)])
    print(kplus)
    print(kminus)
    print(pplus)
    print(pminus)
    # Compute allowed transitions
    valid_X_reactions, valid_Y_reactions = calculate_valid_transitions(n, False)
    
    # Build full Alpha and Beta matrices (N x N) with symbols only at allowed (j,k)
    Alpha_full = sp.zeros(N, N)
    Beta_full  = sp.zeros(N, N)
    
    for (_, _, j, k, _) in valid_X_reactions:
        Alpha_full[j, k] = sp.symbols(f"{prefix}alpha_{j}_{k}")
    for (_, _, j, k, _) in valid_Y_reactions:
        Beta_full[j, k] = sp.symbols(f"{prefix}beta_{j}_{k}")
    
    print("Alpha_full:")
    print(Alpha_full)
    print("\nBeta_full:")
    print(Beta_full)
    
    # Build matrices following numpy approach
    ones_vec = sp.ones(N-1, 1)
    
    # Kp: diag([kplus, 0]) - shape (N, N)
    Kp = sp.diag(*kplus, sp.Integer(0))
    print("Kp")
    print(Kp)
    # Km: vstack([diag(kminus), zeros(1, N-1)]) - shape (N, N-1)
    Km = sp.Matrix.vstack(sp.diag(*kminus), sp.zeros(1, N-1))
    print("Km")
    print(Km)
    # Pp: diag([0, pplus]) - shape (N, N)
    Pp = sp.diag(sp.Integer(0), *pplus)
    print("Pp")
    print(Pp)
    # Pm: vstack([zeros(1, N-1), diag(pminus)]) - shape (N, N-1)
    Pm = sp.Matrix.vstack(sp.zeros(1, N-1), sp.diag(*pminus))
    print("Pm")
    print(Pm)
    print(f"\nKp shape: {Kp.shape}")
    print(f"Km shape: {Km.shape}")
    print(f"Pp shape: {Pp.shape}")
    print(f"Pm shape: {Pm.shape}")
    
    # adjusted_alpha_mat: delete last row from Alpha_full - shape (N-1, N)
    adjusted_alpha_mat = Alpha_full[:-1, :]
    
    # adjusted_beta_mat: delete first row from Beta_full - shape (N-1, N)
    adjusted_beta_mat = Beta_full[1:, :]
    
    print(f"\nadjusted_alpha_mat shape: {adjusted_alpha_mat.shape}")
    print(f"adjusted_beta_mat shape: {adjusted_beta_mat.shape}")
    
    # Da: diag of row sums of Alpha[:-1, 1:] - shape (N-1, N-1)
    alpha_subset = Alpha_full[:-1, 1:]
    Da_diag = [sum(alpha_subset.row(i)) for i in range(N-1)]
    Da = sp.diag(*Da_diag)
    
    # Db: diag of row sums of Beta[1:, :-1] - shape (N-1, N-1)
    beta_subset = Beta_full[1:, :-1]
    Db_diag = [sum(beta_subset.row(i)) for i in range(N-1)]
    Db = sp.diag(*Db_diag)
    
    print(f"\nDa shape: {Da.shape}")
    print(f"Db shape: {Db.shape}")
    
    # U, I: diagonal matrices - shape (N-1, N-1)
    U = sp.diag(*kminus)
    I = sp.diag(*pminus)
    
    # Q: Kp[:-1, :] - shape (N-1, N)
    Q = Kp[:-1, :]
    
    # D: Pp without first row - shape (N-1, N)
    D = Pp[1:, :]
    
    # M and N matrices
    M_mat = U + Da
    N_mat = I + Db
    
    # G and H matrices
    G = Km + adjusted_alpha_mat.T
    H = Pm + adjusted_beta_mat.T
    
    # Compute ODEs using the numpy-style construction
    adot = G * b + H * c - x * (Kp * a) - y * (Pp * a)
    bdot = x * (Q * a) - M_mat * b
    cdot = y * (D * a) - N_mat * c
    
    # For xdot and ydot
    kplus_vec = sp.Matrix([*kplus, sp.Integer(0)])
    pplus_vec = sp.Matrix([sp.Integer(0), *pplus])
    
    xdot = -sum(bdot)
    ydot = -sum(cdot)
    
    return {
        "adot": sp.expand(adot), 
        "bdot": sp.expand(bdot), 
        "cdot": sp.expand(cdot),
        "xdot": sp.expand(xdot), 
        "ydot": sp.expand(ydot),
        "matrices": {
            "Kp": Kp, "Km": Km, "Pp": Pp, "Pm": Pm,
            "Alpha_full": Alpha_full, "Beta_full": Beta_full,
            "adjusted_alpha": adjusted_alpha_mat, "adjusted_beta": adjusted_beta_mat,
            "Da": Da, "Db": Db, "U": U, "I": I,
            "Q": Q, "D": D, "M": M_mat, "N": N_mat,
            "G": G, "H": H
        }
    }

# Example: build symbolic ODEs for n=2 (N=4)
n = 2
odes = build_phos_odes(n)
print("\n\nadot[0]:")
sp.pprint(odes["adot"][0])
odes["adot"]

Matrix([[a_0], [a_1], [a_2], [a_3]])
Matrix([[b_0], [b_1], [b_2]])
Matrix([[c_1], [c_2], [c_3]])
[k^+_0, k^+_1, k^+_2]
[k^-_0, k^-_1, k^-_2]
[p^+_1, p^+_2, p^+_3]
[p^-_1, p^-_2, p^-_3]
00 --> 01 (X), 0 -> 1
00 --> 10 (X), 0 -> 2
01 --> 00 (Y), 1 -> 0
01 --> 11 (X), 1 -> 3
10 --> 00 (Y), 2 -> 0
10 --> 11 (X), 2 -> 3
11 --> 01 (Y), 3 -> 1
11 --> 10 (Y), 3 -> 2
Alpha_full:
Matrix([[0, alpha_0_1, alpha_0_2, 0], [0, 0, 0, alpha_1_3], [0, 0, 0, alpha_2_3], [0, 0, 0, 0]])

Beta_full:
Matrix([[0, 0, 0, 0], [beta_1_0, 0, 0, 0], [beta_2_0, 0, 0, 0], [0, beta_3_1, beta_3_2, 0]])
Kp
Matrix([[k^+_0, 0, 0, 0], [0, k^+_1, 0, 0], [0, 0, k^+_2, 0], [0, 0, 0, 0]])
Km
Matrix([[k^-_0, 0, 0], [0, k^-_1, 0], [0, 0, k^-_2], [0, 0, 0]])
Pp
Matrix([[0, 0, 0, 0], [0, p^+_1, 0, 0], [0, 0, p^+_2, 0], [0, 0, 0, p^+_3]])
Pm
Matrix([[0, 0, 0], [p^-_1, 0, 0], [0, p^-_2, 0], [0, 0, p^-_3]])

Kp shape: (4, 4)
Km shape: (4, 3)
Pp shape: (4, 4)
Pm shape: (4, 3)

adjusted_alpha_mat shape: (3, 4)
adjusted_beta_mat shape: (

Matrix([
[                           -a_0*k^+_0*x + b_0*k^-_0 + beta_1_0*c_1 + beta_2_0*c_2],
[-a_1*k^+_1*x - a_1*p^+_1*y + alpha_0_1*b_0 + b_1*k^-_1 + beta_3_1*c_3 + c_1*p^-_1],
[-a_2*k^+_2*x - a_2*p^+_2*y + alpha_0_2*b_0 + b_2*k^-_2 + beta_3_2*c_3 + c_2*p^-_2],
[                         -a_3*p^+_3*y + alpha_1_3*b_1 + alpha_2_3*b_2 + c_3*p^-_3]])

In [ ]:
n = 2
N = 2**n
a_vars = sp.symbols([f"a_{i}" for i in range(N)])
b_vars = sp.symbols([f"b_{i}" for i in range(N-1)])
c_vars = sp.symbols([f"c_{i}" for i in range(1, N)])
x_var, y_var = sp.symbols("x y")

# Combine all state variables
all_state_vars = list(a_vars) + list(b_vars) + list(c_vars) + [x_var, y_var]

G = odes["G"]; H = odes["H"]; Kp = odes["Kp"]
# Compute Jacobian of adot (each row of adot) with respect to all state variables
print("\n\nJacobian of adot:")
adot_jacobian = odes["adot"].jacobian(all_state_vars)
adot_jacobian





State variables:
[a_0, a_1, a_2, a_3, b_0, b_1, b_2, c_1, c_2, c_3, x, y]


Jacobian of adot:


Matrix([
[-k^+_0*x,                  0,                  0,        0,     k^-_0,         0,         0, beta_1_0, beta_2_0,        0, -a_0*k^+_0,          0],
[       0, -k^+_1*x - p^+_1*y,                  0,        0, alpha_0_1,     k^-_1,         0,    p^-_1,        0, beta_3_1, -a_1*k^+_1, -a_1*p^+_1],
[       0,                  0, -k^+_2*x - p^+_2*y,        0, alpha_0_2,         0,     k^-_2,        0,    p^-_2, beta_3_2, -a_2*k^+_2, -a_2*p^+_2],
[       0,                  0,                  0, -p^+_3*y,         0, alpha_1_3, alpha_2_3,        0,        0,    p^-_3,          0, -a_3*p^+_3]])

In [22]:
bdot_jacobian = odes["bdot"].jacobian(all_state_vars)
bdot_jacobian

Matrix([
[k^+_0*x,       0,       0, 0, -alpha_0_1 - alpha_0_2 - k^-_0,                  0,                  0, 0, 0, 0, a_0*k^+_0, 0],
[      0, k^+_1*x,       0, 0,                              0, -alpha_1_3 - k^-_1,                  0, 0, 0, 0, a_1*k^+_1, 0],
[      0,       0, k^+_2*x, 0,                              0,                  0, -alpha_2_3 - k^-_2, 0, 0, 0, a_2*k^+_2, 0]])

In [23]:
cdot_jacobian = odes["cdot"].jacobian(all_state_vars)
cdot_jacobian

Matrix([
[0, p^+_1*y,       0,       0, 0, 0, 0, -beta_1_0 - p^-_1,                 0,                            0, 0, a_1*p^+_1],
[0,       0, p^+_2*y,       0, 0, 0, 0,                 0, -beta_2_0 - p^-_2,                            0, 0, a_2*p^+_2],
[0,       0,       0, p^+_3*y, 0, 0, 0,                 0,                 0, -beta_3_1 - beta_3_2 - p^-_3, 0, a_3*p^+_3]])

In [25]:
xdot_gradient = sp.Matrix([odes["xdot"]]).jacobian(all_state_vars)
xdot_gradient

Matrix([[-k^+_0*x, -k^+_1*x, -k^+_2*x, 0, alpha_0_1 + alpha_0_2 + k^-_0, alpha_1_3 + k^-_1, alpha_2_3 + k^-_2, 0, 0, 0, -a_0*k^+_0 - a_1*k^+_1 - a_2*k^+_2, 0]])

In [26]:
ydot_gradient = sp.Matrix([odes["ydot"]]).jacobian(all_state_vars)
ydot_gradient

Matrix([[0, -p^+_1*y, -p^+_2*y, -p^+_3*y, 0, 0, 0, beta_1_0 + p^-_1, beta_2_0 + p^-_2, beta_3_1 + beta_3_2 + p^-_3, 0, -a_1*p^+_1 - a_2*p^+_2 - a_3*p^+_3]])

In [27]:
import numpy as np
from functools import reduce

# Example matrices
A = np.array([[1, 0],
              [0, 1]])
B = np.array([[0, 1],
              [1, 0]])
C = np.array([[1, 1],
              [1, -1]])

# Compute Kronecker product of all
result = reduce(np.kron, [A, B, C])

print(result)


[[ 0  0  1  1  0  0  0  0]
 [ 0  0  1 -1  0  0  0  0]
 [ 1  1  0  0  0  0  0  0]
 [ 1 -1  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  1  1]
 [ 0  0  0  0  0  0  1 -1]
 [ 0  0  0  0  1  1  0  0]
 [ 0  0  0  0  1 -1  0  0]]


In [29]:
import numpy as np

A = np.array([[1, 0],
              [0, 1]])
B = np.array([[0, 1],
              [1, 0]])

cols = B.flatten(order='F')               # -> array([0, 1, 1, 0])
result = np.hstack([A[:, int(i) : int(i)+1] for i in cols])
print(result)


[[1 0 0 1]
 [0 1 1 0]]


In [50]:
import numpy as np

# Create a sample NumPy array
arr = np.array([[1, 2, 3],
                [4, 5, 6],
                [7, 8, 9],
                [10, 11, 12]])

# Remove the last row using slicing
arr_without_last_row = arr[:-1, :]

print(arr_without_last_row)

[[1 2 3]
 [4 5 6]
 [7 8 9]]


In [53]:
import numpy as np

A = np.array([[1, 0], [0, 1]])
B = np.array([[0, 1], [1, 0]])
# C = np.array([[5, 1], [1, 6]])
result = np.hstack([A, B])
print(result)

[[1 0 0 1]
 [0 1 1 0]]


In [55]:
import numpy as np

A = np.array([[1, 0], [0, 1]])
B = np.array([[0, 1], [1, 0]])

result = np.vstack([A, B])
# Result: array([[1, 0],
#                [0, 1],
#                [0, 1],
#                [1, 0]])
print(result)

[[1 0]
 [0 1]
 [0 1]
 [1 0]]


In [ ]:
attempt_num = 10
for i in range(attempt_num + 1):
    print(i / attempt_num)

0.0
0.1
0.2
0.3
0.4
0.5
0.6
0.7
0.8
0.9
1.0
